# Calculo de Probabilidade de Viagem

Este notebook calcula a probabilidade de uma pessoa infectada viajar para um determinadas cidade a qual possui conexão.

In [1]:
import pandas as pd
import numpy as np

In [29]:
df_municipio = pd.read_csv("data/integrado/municipio.csv")
df_fluxo_aereo = pd.read_csv("data/integrado/fluxo_aereo.csv")
df_fluxo_rodoviario = pd.read_csv("data/integrado/fluxo_rodoviario.csv")

In [9]:
df_fluxo_aereo

,cod_mun_origem,cod_mun_destino,total_passagens_2019,total_passagens_2020,tempo_medio_2019,tempo_medio_2020
0,3550308,3304557,1027294.0,455330.0,59.0,58.0
1,3304557,3550308,1026248.0,455700.0,65.0,65.0
2,4314902,3550308,626155.0,309355.0,98.0,96.0
3,3550308,4314902,626063.0,308367.0,101.0,100.0
4,3550308,2927408,518955.0,286796.0,138.0,137.0
...,...,...,...,...,...,...
6744,2304400,1303403,NaN,1.0,NaN,-2.0
6745,5105150,3106200,NaN,4.0,NaN,-2.0
6746,1200401,5105150,NaN,1.0,NaN,-2.0
6747,3106200,5105150,NaN,2.0,NaN,-2.0


In [12]:
df_fluxo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_origem == 2611606][['cod_mun_destino', 'total_passagens_2020']]
lista_fluxo = list(df_fluxo.itertuples(index=False, name=None))

In [35]:
d1 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_a == 2611606)][['cod_mun_b', 'freq_saida_2016']].rename(columns={"cod_mun_b": "cod_mun"})
d2 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_b == 2611606)][['cod_mun_a', 'freq_saida_2016']].rename(columns={"cod_mun_a": "cod_mun"})
d1.append(d2, ignore_index=True).sort_values(by=['freq_saida_2016'])

,cod_mun,freq_saida_2016
102,5211503,1.0
197,1100205,1.0
221,1100122,1.0
222,1100189,1.0
96,3554102,1.0
...,...,...
176,2608453,990.0
155,2608909,1051.0
255,2605202,1214.0
177,2604007,1561.0


In [48]:
def info_cidade(codigo_municipio, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario):
    
    #Consultando valores nos dataframe a partir do código da cidade
    total_populacao = df_municipio[df_municipio.cod_mun == codigo_municipio].populacao_2021
    
    df_aereo = df_fluxo_aereo[df_fluxo_aereo.cod_mun_origem == codigo_municipio][['cod_mun_destino', 'total_passagens_2020']].rename(columns={"cod_mun_destino": "cod_mun"})
    df_rod_1 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_a == codigo_municipio)][['cod_mun_b', 'freq_saida_2016']].rename(columns={"cod_mun_b": "cod_mun"})
    df_rod_2 = df_fluxo_rodoviario[(df_fluxo_rodoviario.cod_mun_b == codigo_municipio)][['cod_mun_a', 'freq_saida_2016']].rename(columns={"cod_mun_a": "cod_mun"})
    df_rod = df_rod_1.append(df_rod_2, ignore_index=True).sort_values(by=['freq_saida_2016'])
    
    df_fluxo = df_aereo.merge(df_rod[['cod_mun', 'freq_saida_2016']], how='outer', left_on='cod_mun', right_on='cod_mun')
    df_fluxo.fillna(0, inplace = True)

    lista_fluxo = list(df_fluxo.itertuples(index=False, name=None))
    
    return (total_populacao, lista_fluxo)

In [52]:
FATOR_RODOV = 20 *365

## Calculando as probabilidade de viajar e estar infectado
total_populacao, lista_fluxo= info_cidade(2611606, df_municipio, df_fluxo_aereo, df_fluxo_rodoviario)
fluxo_total = sum(j for i, j, k in lista_fluxo)
fluxo_total += sum(k for i, j, k in lista_fluxo) * FATOR_RODOV ##Valor que tem que mudar, essa é minha estimativa

numero_infectados = 10
prob_infectado = numero_infectados / total_populacao

prob_final = []
for codigo, fluxo_aereo, fluxo_rodo in lista_fluxo:
    print(f"{codigo} - {fluxo_aereo} - {fluxo_rodo}")

    fluxo = fluxo_aereo + fluxo_rodo * FATOR_RODOV
    prob_viajar = fluxo / fluxo_total

    prob_final.append((codigo, prob_viajar))

3550308 - 287035.0 - 38.0
2304400 - 71275.0 - 91.0
3304557 - 97299.0 - 32.0
5300108 - 88821.0 - 26.0
2927408 - 41944.0 - 38.0
3106200 - 22107.0 - 14.0
4314902 - 30038.0 - 0.0
2611101 - 12746.0 - 63.0
3509502 - 21587.0 - 19.0
2605459 - 20384.0 - 0.0
1302603 - 31506.0 - 0.0
2211001 - 11989.0 - 66.0
1501402 - 19551.0 - 19.0
4106902 - 20109.0 - 7.0
3205309 - 12621.0 - 6.0
5208707 - 18550.0 - 43.0
2111300 - 12485.0 - 21.0
5103403 - 17618.0 - 16.0
2800308 - 6550.0 - 49.0
4108304 - 5666.0 - 2.0
4208203 - 10841.0 - 0.0
4205407 - 11627.0 - 0.0
2307304 - 4538.0 - 28.0
5002704 - 9983.0 - 7.0
1100205 - 6497.0 - 1.0
3543402 - 0.0 - 16.0
4113700 - 0.0 - 0.0
3549805 - 0.0 - 16.0
3170206 - 5779.0 - 27.0
2704302 - 1529.0 - 136.0
2408102 - 2217.0 - 72.0
1721000 - 3557.0 - 14.0
1600303 - 2582.0 - 0.0
2105302 - 0.0 - 6.0
4209102 - 0.0 - 0.0
4305108 - 0.0 - 0.0
4115200 - 3228.0 - 0.0
1506807 - 1344.0 - 0.0
1504208 - 2106.0 - 6.0
4204202 - 0.0 - 0.0
1400100 - 2439.0 - 0.0
2408003 - 0.0 - 76.0
1200401 - 1433

In [53]:
prob_final.sort(key=lambda x:-x[1])

In [54]:
for cod_mun, prob in prob_final:
    nome = df_municipio.loc[df_municipio.cod_mun==cod_mun, 'nome_mun'].values[0]
    print(f"{nome} - {prob}")

Paudalho - 0.06237234686233251
Carpina - 0.06108107493858284
Escada - 0.04750315501309389
Limoeiro - 0.041125054298815224
Lagoa do Carro - 0.038738157712490076
Caruaru - 0.03513824810688494
Ribeirão - 0.03462956522783204
Vitória de Santo Antão - 0.03455130632336236
Gameleira - 0.03232092754597657
Palmares - 0.0320470213803327
Gravatá - 0.023203765175259208
João Pessoa - 0.018861778908517698
Arcoverde - 0.018664748716017948
Bezerros - 0.01862561926378311
Sirinhaém - 0.014712674040299262
Campina Grande - 0.013343143212079916
Rio Formoso - 0.0132257548553754
Goiana - 0.013069237046436047
Pombos - 0.012873589785261855
Belo Jardim - 0.012873589785261855
Tracunhaém - 0.0116997062182167
Nazaré da Mata - 0.011621447313747023
Pesqueira - 0.011504058957042509
Chã Grande - 0.010682340460110901
Serra Talhada - 0.010095398676588324
Custódia - 0.0091562918229522
Camocim de São Félix - 0.008491091134959947
Garanhuns - 0.008491091134959947
Sairé - 0.008491091134959947
Lagoa de Itaenga - 0.008217184969